In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string

In [2]:
df = pd.read_csv('datasets/biology.csv')
df.head()

,id,title,content,tags
0,1,What is the criticality of the ribosome bindin...,"<p>In prokaryotic translation, how critical fo...",ribosome binding-sites translation synthetic-b...
1,2,How is RNAse contamination in RNA based experi...,<p>Does anyone have any suggestions to prevent...,rna biochemistry
2,3,Are lymphocyte sizes clustered in two groups?,<p>Tortora writes in <em>Principles of Anatomy...,immunology cell-biology hematology
3,4,How long does antibiotic-dosed LB maintain goo...,<p>Various people in our lab will prepare a li...,cell-culture
4,5,Is exon order always preserved in splicing?,<p>Are there any cases in which the splicing m...,splicing mrna spliceosome introns exons


In [3]:
# def tag_in_text(infile):
    
#     df = pd.read_csv('datasets/%s' % infile)
    
#     df['clean content'] = df['content'].apply(cleanhtml)
#     df['clean title'] = df['title'].apply(cleanhtml)
    
#     tags_in_text = []

#     for i in range(len(df.values)):
#         tags = df['tags'][i].split()
#         for j in range(len(tags)):
#             if tags[j] in df['clean content'][i]:
#                 tags_in_text.append(1)
#             elif tags[j] in df['clean title'][i]:
#                 tags_in_text.append(1)
#             else:
#                 tags_in_text.append(0)
#     return np.mean(tags_in_text)

In [2]:
def create_clean_columns(infile):
    df = pd.read_csv('datasets/%s' % infile)
    
    df['clean content'] = df['content'].apply(cleanhtml)
    df['clean title'] = df['title'].apply(cleanhtml)
    return df

In [3]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', '')
    cleantext = cleantext.translate(string.maketrans("",""), string.punctuation)
    cleantext = cleantext.lower()
    return cleantext

In [4]:
def do_tfidf(df):
    tvec = TfidfVectorizer(stop_words='english')
    tvec.fit(df['clean content'])

    words_df = pd.DataFrame(tvec.transform(df['clean content']).todense(),
                   columns=tvec.get_feature_names())
    return words_df

In [5]:
def get_best_word(df, index):
    row = df.iloc[index, :]
    row_tuple = zip(row.index, row.values)
    return max(row_tuple, key=lambda x: x[1])

In [6]:
def get_list_of_tags(df):
    list_of_tags = []

    for i in range(len(df)):
        list_of_tags.append(get_best_word(df, i))
    better_list_of_tags = []

    for i in range(len(list_of_tags)):
        better_list_of_tags.append(list_of_tags[i][0])
    
    return better_list_of_tags

In [ ]:
def get_best_word(df, index):
    row = df.iloc[index, :]
    row_tuple = zip(row.index, row.values)
    return max(row_tuple, key=lambda x: x[1])

In [19]:
def get_list_of_tags(df):
    list_of_tags = []

#     for i in range(len(df)):
#         list_of_tags.append(get_best_word(df, i))
#     better_list_of_tags = []

    list_of_tags.append(df.apply(lambda x: get_best_word(df, df.index[x])))

    for i in range(len(list_of_tags)):
        better_list_of_tags.append(list_of_tags[i][0])
    
    return better_list_of_tags

In [7]:
def create_prediction_column(df, words_df):
    df['predicted_tag'] = get_list_of_tags(words_df)

In [8]:
df = create_clean_columns('test.csv')

In [9]:
print df.shape
df.head()

(81926, 5)


,id,title,content,clean content,clean title
0,1,What is spin as it relates to subatomic partic...,<p>I often hear about subatomic particles havi...,i often hear about subatomic particles having ...,what is spin as it relates to subatomic particles
1,2,What is your simplest explanation of the strin...,<p>How would you explain string theory to non ...,how would you explain string theory to non phy...,what is your simplest explanation of the strin...
2,3,"Lie theory, Representations and particle physics",<p>This is a question that has been posted at ...,this is a question that has been posted at man...,lie theory representations and particle physics
3,7,Will Determinism be ever possible?,<p>What are the main problems that we need to ...,what are the main problems that we need to sol...,will determinism be ever possible
4,9,Hamilton's Principle,<p>Hamilton's principle states that a dynamic ...,hamiltons principle states that a dynamic syst...,hamiltons principle


In [12]:
words_df = do_tfidf(df)
words_df.head()

MemoryError: 

In [24]:
create_prediction_column(df, words_df)

IndexError: ('arrays used as indices must be of integer (or boolean) type', u'occurred at index 00')

In [37]:
df.head()

,id,title,content,tags,clean content,clean title,predicted_tag
0,1,What is the right approach to write the spin c...,<p>Imagine programming a 3 wheel soccer robot....,soccer control,imagine programming a 3 wheel soccer robot wha...,what is the right approach to write the spin c...,addedvalue
1,2,How can I modify a low cost hobby servo to run...,"<p>I've got some hobby servos (<a href=""http:/...",control rcservo,ive got some hobby servos power hd 1501mgs and...,how can i modify a low cost hobby servo to run...,1501mgs
2,3,What useful gaits exist for a six legged robot...,"<p><a href=""http://www.oricomtech.com/projects...",gait walk,httpwwworicomtechcomprojectslegtimehtm lists t...,what useful gaits exist for a six legged robot...,andripplecan
3,4,Good Microcontrollers/SOCs for a Robotics Project,<p>I am looking for a starting point for my pr...,microcontroller arduino raspberry-pi,i am looking for a starting point for my proje...,good microcontrollerssocs for a robotics project,systems
4,5,Nearest-neighbor data structure for non-Euclid...,<p>I'm trying to implement a nearest-neighbor ...,motion-planning rrt,im trying to implement a nearestneighbor struc...,nearestneighbor data structure for noneuclidea...,noneuclidean


In [42]:
df.shape

(2771, 7)

In [51]:
sub_df = df[['id', 'predicted_tag']]

In [54]:
sub_df.dtypes

"id"       int64
"tags"    object
dtype: object

In [52]:
sub_df.columns = ['id', 'tags']
sub_df.head()

,"""id""","""tags"""
0,1,addedvalue
1,2,1501mgs
2,3,andripplecan
3,4,systems
4,5,noneuclidean


In [56]:
sub_df.to_csv('submission.csv', encoding='utf-8', index=False)